<a href="https://colab.research.google.com/github/DineshKatravath/da6401_assignment1/blob/main/DA6401_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>